In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pickle

In [3]:
data_dir = 'E:\EVILDOLL\PYTHON PROJECTS\TENSER FLOW\Transfer Control\image_data'
categories = ['manipuri','bharatanatyam','odissi','kathakali','kathak','sattriya','kuchipudi','mohiniyattam']
data = []

In [4]:
img_size = 224
def make_data():
    for category in categories:
        path = os.path.join(data_dir, category)
        label = categories.index(category)
        for img_name in os.listdir(path):
            image_path = os.path.join(path, img_name)
            image = cv2.imread(image_path)
            try:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image,(img_size,img_size))
                image = np.array(image, dtype=np.float32)
                data.append([image,label])
            except Exception as e:
                pass  
            
make_data() 

In [5]:
pik = open('data.pickle', 'wb')
pickle.dump(data, pik)
pik.close()

In [6]:
def load_data():
    pick = open('data.pickle','rb')
    data = pickle.load(pick)
    pick.close()  
    np.random.shuffle(data)
    features = []
    labels = []
    for img,label in data:
        features.append(img)
        labels.append(label)
        
    features = np.array(features, dtype = np.float32)
    labels = np.array(labels)
    features = features/255.0
    return [features, labels]

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
features, labels = load_data()
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, random_state=30)

In [9]:
import keras
import wandb
from wandb.keras import WandbCallback
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.applications.resnet50 import ResNet50,decode_predictions, preprocess_input

In [10]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [11]:
X_train = X_train/255.
X_test = X_test/255.

In [12]:
simple_model = Sequential()
simple_model.add(Flatten())
simple_model.add(Dense(8, activation = 'sigmoid'))
simple_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [15]:
simple_model.fit(X_train, y_train, epochs = 50, validation_data = (X_test,y_test))

Train on 327 samples, validate on 37 samples
Epoch 1/50
327/327 [==============================] - 2s 6ms/step - loss: 0.8956 - accuracy: 0.8991 - val_loss: 1.6604 - val_accuracy: 0.4595
Epoch 2/50
327/327 [==============================] - 1s 4ms/step - loss: 0.8838 - accuracy: 0.9052 - val_loss: 1.7110 - val_accuracy: 0.4054
Epoch 3/50
327/327 [==============================] - 1s 3ms/step - loss: 0.8779 - accuracy: 0.9113 - val_loss: 1.7351 - val_accuracy: 0.3784
Epoch 4/50
327/327 [==============================] - 1s 2ms/step - loss: 0.8653 - accuracy: 0.9113 - val_loss: 1.6731 - val_accuracy: 0.4324
Epoch 5/50
327/327 [==============================] - 1s 2ms/step - loss: 0.8522 - accuracy: 0.9113 - val_loss: 1.7110 - val_accuracy: 0.4054
Epoch 6/50
327/327 [==============================] - 1s 2ms/step - loss: 0.8361 - accuracy: 0.9266 - val_loss: 1.7320 - val_accuracy: 0.4054
Epoch 7/50
327/327 [==============================] - 1s 2ms/step - loss: 0.8336 - accuracy: 0.9205 - v

In [14]:
resnet_model = ResNet50(weights = 'imagenet')

In [16]:
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [17]:
X_train_pre = preprocess_input(X_train)
X_test_pre = preprocess_input(X_test)

In [18]:
last_layer = resnet_model.get_layer('avg_pool')

resnet_layers = keras.Model(input = resnet_model.inputs, outputs = last_layer.output)
resnet_layers.summary()

C:\Users\Admin\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("av..., inputs=[<tf.Tenso...)`
  This is separate from the ipykernel package so we can avoid doing imports until


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

bn5b_branch2a (BatchNormalizati (None, 7, 7, 512)    2048        res5b_branch2a[0][0]             
__________________________________________________________________________________________________
activation_44 (Activation)      (None, 7, 7, 512)    0           bn5b_branch2a[0][0]              
__________________________________________________________________________________________________
res5b_branch2b (Conv2D)         (None, 7, 7, 512)    2359808     activation_44[0][0]              
__________________________________________________________________________________________________
bn5b_branch2b (BatchNormalizati (None, 7, 7, 512)    2048        res5b_branch2b[0][0]             
__________________________________________________________________________________________________
activation_45 (Activation)      (None, 7, 7, 512)    0           bn5b_branch2b[0][0]              
__________________________________________________________________________________________________
res5b_bran

In [19]:
X_train_features = resnet_layers.predict(X_train_pre)
X_test_features = resnet_layers.predict(X_test_pre)

In [20]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=40, verbose=1, mode='auto')

In [21]:
feature_model = Sequential()
feature_model.add(Dense(8, activation = 'sigmoid'))
feature_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'],callbacks=[checkpoint,early])

In [23]:
feature_model.fit(X_train_features, y_train, epochs = 100, validation_data = (X_test_features, y_test))

Train on 327 samples, validate on 37 samples
Epoch 1/100
327/327 [==============================] - 0s 323us/step - loss: 2.0767 - accuracy: 0.1131 - val_loss: 2.1075 - val_accuracy: 0.1622
Epoch 2/100
327/327 [==============================] - 0s 281us/step - loss: 2.0776 - accuracy: 0.1346 - val_loss: 2.1254 - val_accuracy: 0.0811
Epoch 3/100
327/327 [==============================] - 0s 360us/step - loss: 2.0784 - accuracy: 0.1162 - val_loss: 2.1242 - val_accuracy: 0.1351
Epoch 4/100
327/327 [==============================] - 0s 418us/step - loss: 2.0773 - accuracy: 0.1346 - val_loss: 2.1021 - val_accuracy: 0.1351
Epoch 5/100
327/327 [==============================] - 0s 351us/step - loss: 2.0777 - accuracy: 0.1346 - val_loss: 2.0874 - val_accuracy: 0.1892
Epoch 6/100
327/327 [==============================] - 0s 357us/step - loss: 2.0772 - accuracy: 0.1315 - val_loss: 2.0938 - val_accuracy: 0.1892
Epoch 7/100
327/327 [==============================] - 0s 384us/step - loss: 2.0762 -

327/327 [==============================] - 0s 143us/step - loss: 2.0740 - accuracy: 0.1346 - val_loss: 2.1055 - val_accuracy: 0.1351
Epoch 57/100
327/327 [==============================] - 0s 131us/step - loss: 2.0755 - accuracy: 0.1346 - val_loss: 2.1068 - val_accuracy: 0.1351
Epoch 58/100
327/327 [==============================] - 0s 119us/step - loss: 2.0751 - accuracy: 0.1193 - val_loss: 2.1070 - val_accuracy: 0.0811
Epoch 59/100
327/327 [==============================] - 0s 122us/step - loss: 2.0761 - accuracy: 0.1346 - val_loss: 2.1096 - val_accuracy: 0.0811
Epoch 60/100
327/327 [==============================] - 0s 116us/step - loss: 2.0748 - accuracy: 0.1346 - val_loss: 2.1119 - val_accuracy: 0.0811
Epoch 61/100
327/327 [==============================] - 0s 104us/step - loss: 2.0751 - accuracy: 0.1346 - val_loss: 2.1121 - val_accuracy: 0.0811
Epoch 62/100
327/327 [==============================] - 0s 107us/step - loss: 2.0750 - accuracy: 0.1346 - val_loss: 2.1109 - val_accuracy

In [24]:
new_model = Sequential()
new_model.add(resnet_layers)
new_model.add(Dense(8, activation = 'sigmoid'))
new_model.layers[0].trainable = False
new_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
new_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 2048)              23587712  
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 16392     
Total params: 23,604,104
Trainable params: 16,392
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
new_model.fit(X_train_pre,y_train, epochs = 10, validation_data = (X_test_pre,y_test),callbacks=[checkpoint,early])

Train on 327 samples, validate on 37 samples
Epoch 1/10
327/327 [==============================] - 102s 312ms/step - loss: 2.0728 - accuracy: 0.1254 - val_loss: 2.1494 - val_accuracy: 0.1622
Epoch 2/10
327/327 [==============================] - 100s 305ms/step - loss: 2.0739 - accuracy: 0.1254 - val_loss: 2.1548 - val_accuracy: 0.1622
Epoch 3/10
327/327 [==============================] - 99s 302ms/step - loss: 2.0721 - accuracy: 0.1254 - val_loss: 2.1580 - val_accuracy: 0.1622
Epoch 4/10
327/327 [==============================] - 99s 303ms/step - loss: 2.0716 - accuracy: 0.1254 - val_loss: 2.1618 - val_accuracy: 0.1622
Epoch 5/10
 32/327 [=>............................] - ETA: 1:26 - loss: 2.0940 - accuracy: 0.1562

In [ ]:
new_model.layers[0].trainable = True

for layer in new_model.layers[0].layers[:-11]:
    layer.trainable = False
for layer in new_model.layers[0].layers[-11:]:
    layer.trainable = True
new_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
new_model.fit(X_train_pre,y_train, epochs = 10, validation_data = (X_test_pre,y_test))